In [2]:
import pandas as pd
import numpy as np

In [3]:
drug_deaths = pd.read_csv(r'C:\Users\lenovo\Downloads\Accidental_Drug_Related_Deaths_2012-2024.csv')

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
drug_deaths.head()

,Date,Date Type,Age,Sex,Race,Ethnicity,Residence City,Residence County,Residence State,Injury City,Injury County,Injury State,Injury Place,Description of Injury,Death City,Death County,Death State,Location,Location if Other,Cause of Death,Manner of Death,Other Significant Conditions,Heroin,Heroin death certificate (DC),Cocaine,Fentanyl,Fentanyl Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Meth/Amphetamine,Amphet,Tramad,Hydromorphone,Morphine (Not Heroin),Xylazine,Gabapentin,Opiate NOS,Heroin/Morph/Codeine,Other Opioid,Any Opioid,Other,ResidenceCityGeo,InjuryCityGeo,DeathCityGeo
0,01/02/2023,Date of death,65.0,Male,White,"No, not Spanish/Hispanic/Latino",BRIDGEPORT,FAIRFIELD,CT,BRIDGEPORT,FAIRFIELD,NaN,Other,Substance use,NaN,NaN,NaN,NaN,NaN,Acute Intoxication by the Combined Effects of ...,Accident,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,"BRIDGEPORT, CT\n(41.179195, -73.189476)","BRIDGEPORT, CT\n(41.179195, -73.189476)","CT\n(41.57350273, -72.738305908)"
1,01/18/2023,Date of death,42.0,Male,White,"No, not Spanish/Hispanic/Latino",WATERBURY,NEW HAVEN,CT,FARMINGTON,HARTFORD,CT,Home,Ethanol and Fentanyl Use,NaN,NaN,NaN,NaN,NaN,Combined Effects of Fentanyl and Ethanol Toxicity,Accident,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,"WATERBURY, CT\n(41.5542609, -73.0430692)","FARMINGTON, CT\n(41.726536, -72.8250564)","CT\n(41.57350273, -72.738305908)"
2,01/26/2022,Date of death,34.0,Male,White,NaN,BROOKFIELD,FAIRFIELD,CT,BROOKFIELD,FAIRFIELD,CT,Home,Substance Abuse,NaN,NaN,NaN,Decedent’s Home,NaN,Acute Intoxication due to the Combined Effects...,Accident,NaN,Y,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,Y,NaN,Y,NaN,"BROOKFIELD, CT\n(41.46736000000004, -73.398259...","BROOKFIELD, CT\n(41.46736000000004, -73.398259...","CT\n(41.57350273000003, -72.73830590799997)"
3,02/04/2023,Date of death,34.0,Male,White,NaN,NaN,NaN,NaN,NEW HAVEN,NEW HAVEN,CT,Other,Substance Use,NaN,NaN,NaN,NaN,NaN,Acute Fentanyl Intoxication with Recent Cocain...,Accident,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,"CT\n(41.57350273, -72.738305908)","NEW HAVEN, CT\n(41.3082517, -72.9241605)","CT\n(41.57350273, -72.738305908)"
4,02/18/2022,Date of death,43.0,Male,White,NaN,NORWICH,NEW LONDON,CT,NORWICH,NEW LONDON,CT,Public Recreation Area,Drug Use,NaN,NaN,NaN,Other (Specify),Mohegan Park,Acute Cocaine and Fentanyl Intoxication,Accident,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,"NORWICH, CT\n(41.526760000000024, -72.07567999...","NORWICH, CT\n(41.526760000000024, -72.07567999...","CT\n(41.57350273000003, -72.73830590799997)"


### Data Cleansing

#### capitalizing caps rows

In [6]:
# 1. converting nan values to blank as nan values can't be capitalised ->  fillna('')
# 2. creating a lambda function that capitalizes each row in specified columns ->  lambda x: x.capitalize()

drug_deaths[['Residence City', 'Residence County', 'Injury City', 'Injury County']] = drug_deaths[['Residence City', 'Residence County', 'Injury City', 'Injury County']].fillna('').map(lambda x: x.capitalize())

#### recreating ethnicity column

In [7]:
# ethnicity column values are inconsistent, overlapping, and confusing
drug_deaths['Ethnicity'].value_counts()

Ethnicity
No, not Spanish/Hispanic/Latino                 1409
Hispanic                                         972
Yes, other Spanish/Hispanic/Latino               307
Spanish/Hispanic/Latino                          267
Other Spanish/Hispanic/Latino                    254
Yes, Puerto Rican                                 85
Not Spanish/Hispanic/Latino                       56
Yes, Other Spanish/Hispanic/Latino (Specify)      26
Unknown                                           24
Puerto Rican                                      19
Yes, Mexican, Mexican American, Chicano            8
Mexican, Mexican American, Chicano                 4
Cuban                                              2
Yes, Cuban                                         1
n                                                  1
Name: count, dtype: int64

In [8]:
# creating a lambda function to make the column values significantly more readible

ethnicity_lambda = lambda x: (
    'Non-Hispanic/Latino' if (
        'no, not' in str(x).lower() or 
        'not spanish/hispanic/latino' in str(x).lower()
    ) 
    else 'Hispanic/Latino' if (
        'yes,' in str(x).lower() or 
        'hispanic' in str(x).lower() or 
        'latino' in str(x).lower() or 
        'puerto rican' in str(x).lower() or 
        'mexican' in str(x).lower() or 
        'chicano' in str(x).lower() or 
        'cuban' in str(x).lower()
    ) 
    else 'Unknown/Missing'
)

drug_deaths['Ethnicity_New'] = drug_deaths['Ethnicity'].apply(ethnicity_lambda)

In [9]:
# deleting the original ethnicity column

drug_deaths.drop(columns=('Ethnicity'), inplace=True)

#### getting rid of some columns

In [10]:
# the specified columns don't hold usable data but slow down the fetching

drug_deaths.drop(columns=(['ResidenceCityGeo', 'InjuryCityGeo', 'DeathCityGeo']), inplace=True)

#### checking for duplicates

In [10]:
# the table doesn't have any duplicates

drug_deaths[drug_deaths.duplicated(keep=False)]

,Date,Date Type,Age,Sex,Race,Residence City,Residence County,Residence State,Injury City,Injury County,Injury State,Injury Place,Description of Injury,Death City,Death County,Death State,Location,Location if Other,Cause of Death,Manner of Death,Other Significant Conditions,Heroin,Heroin death certificate (DC),Cocaine,Fentanyl,Fentanyl Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Meth/Amphetamine,Amphet,Tramad,Hydromorphone,Morphine (Not Heroin),Xylazine,Gabapentin,Opiate NOS,Heroin/Morph/Codeine,Other Opioid,Any Opioid,Other,ResidenceCityGeo,InjuryCityGeo,DeathCityGeo,Ethnicity_New


#### modifying categorical values in Race column

In [11]:
# for example, categorical values like 'Black' and 'Black or African American' can be combined

drug_deaths.value_counts('Race')

Race
White                                                       10802
Black or African American                                    1027
Black                                                         809
Unknown                                                        63
Other                                                          52
Other (Specify)                                                46
Asian Indian                                                   31
Asian, Other                                                   24
Other Asian                                                    12
American Indian or Alaska Native                                6
Asian/Indian                                                    5
Asian                                                           4
Other (Specify) Puerto Rican                                    3
white                                                           3
Chinese                                                         2
Korea

In [12]:
drug_deaths['Race'] = drug_deaths['Race'].replace(
    {
    'Black' : 'Black or African American',
    'Other' : 'Other (Specify)',
    'Other (Specify) portuguese, Cape Verdean' : 'Other',
    'Other (Specify)' : 'Other',
    'Other (Specify) Puerto Rican' : 'Puerto Rican',
    'white' : 'White',
    'Native American, Other' : 'Other',
    'Black or African American / American Indian Lenni Lenape' : 'Other',
    'Other (Specify) Haitian' : 'Haitian',
    'Hawaiian' : 'Other',
    'Other Asian (Specify)' : 'Asian',
    'Asian Indian' : 'Asian',
    'Asian, Other' : 'Other',
    'Other, Asian' : 'Other',
    'Asian/Indian' : 'Other',
    'Unknown' : 'Other',
    'Other (Specify) portugese, Cape Verdean' : 'Other',
    'Other (Specify)' : 'Other',
    'Other Asian' : 'Asian',
    'Chinese' : 'Asian', 
    'Japanese' : 'Asian',
    'Korean' : 'Asian'
    }
)

In [13]:
drug_deaths.value_counts('Race')

Race
White                               10805
Black or African American            1836
Other                                 142
Asian                                  52
Other (Specify)                        52
American Indian or Alaska Native        6
Puerto Rican                            3
Haitian                                 1
Name: count, dtype: int64

### EDA

#### how many deaths occured each year?

In [14]:
# 1. converting 'date' column data type to datetime
# 2. extracting the year from it
# 3. grouping by year
# 4. checking how many deaths occured and sorting in descending order

drug_deaths.groupby(pd.to_datetime(drug_deaths['Date'], format='%m/%d/%Y').dt.year).size().sort_values(ascending=False)

Date
2021    1524
2022    1452
2020    1374
2023    1327
2019    1200
2017    1038
2018    1017
2024     982
2016     917
2015     729
2014     558
2013     490
2012     355
dtype: int64

#### of those who died, how many were males / females?

In [15]:
drug_deaths.groupby('Sex').size()

Sex
Female     3380
Male       9571
Unknown       2
X             1
dtype: int64

In [16]:
drug_deaths['Sex'].value_counts()

Sex
Male       9571
Female     3380
Unknown       2
X             1
Name: count, dtype: int64

#### of those who died, what is the average age for males / females?

In [17]:
# not outputting 'Unknown', 'X' to not to skew the results

drug_deaths[~drug_deaths['Sex'].isin(['Unknown', 'X'])].groupby('Sex')['Age'].mean().round()

Sex
Female    45.0
Male      44.0
Name: Age, dtype: float64

#### how many males of black or african american race below the age of 30 died in 2018?

In [21]:
drug_deaths[
    (drug_deaths['Sex'] == 'Male') & 
    (drug_deaths['Race'] == 'Black or African American') & 
    (drug_deaths['Age'] < 30) & 
    ((pd.to_datetime(drug_deaths['Date'], format='%m/%d/%Y').dt.year) == 2018)
] 

,Date,Date Type,Age,Sex,Race,Residence City,Residence County,Residence State,Injury City,Injury County,Injury State,Injury Place,Description of Injury,Death City,Death County,Death State,Location,Location if Other,Cause of Death,Manner of Death,Other Significant Conditions,Heroin,Heroin death certificate (DC),Cocaine,Fentanyl,Fentanyl Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Meth/Amphetamine,Amphet,Tramad,Hydromorphone,Morphine (Not Heroin),Xylazine,Gabapentin,Opiate NOS,Heroin/Morph/Codeine,Other Opioid,Any Opioid,Other,ResidenceCityGeo,InjuryCityGeo,DeathCityGeo,Ethnicity_New
3045,11/28/2018,Date of death,26.0,Male,Black or African American,Enfield,Hartford,CT,Enfield,Hartford,NaN,Residence,Substance Abuse,ENFIELD,HARTFORD,CT,Residence,NaN,Acute Fentanyl Intoxication,Accident,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,"ENFIELD, CT\n(41.976501, -72.591985)","ENFIELD, CT\n(41.976501, -72.591985)","Enfield, CT\n(41.976501, -72.591985)",Unknown/Missing
3245,09/15/2018,Date of death,22.0,Male,Black or African American,Uncasville,New london,CT,Uncasville,New london,NaN,Residence,Ingested drugs,UNCASVILLE,NEW LONDON,CT,Residence,NaN,Acute Methadone and Diazepam Intoxication,Accident,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,"UNCASVILLE, CT\n(41.434399, -72.110298)","UNCASVILLE, CT\n(41.434399, -72.110298)","Uncasville, CT\n(41.434399, -72.110298)",Unknown/Missing
4060,03/12/2018,Date of death,23.0,Male,Black or African American,Hamden,New haven,CT,New haven,New haven,NaN,Store or Shopping Area,Substance Abuse,NEW HAVEN,NEW HAVEN,CT,Hospital,NaN,Acute Intoxication due to the Combined Effects...,Accident,NaN,Y,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,Y,NaN,"HAMDEN, CT\n(41.382918, -72.907743)","NEW HAVEN, CT\n(41.308252, -72.924161)","New Haven, CT\n(41.308252, -72.924161)",Unknown/Missing
4350,04/20/2018,Date of death,27.0,Male,Black or African American,West haven,New haven,CT,West haven,New haven,NaN,Residence,Substance Abuse,NEW HAVEN,NEW HAVEN,CT,Hospital,NaN,Acute Intoxication From the Combined Effects o...,Accident,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,"WEST HAVEN, CT\n(41.272336, -72.949817)","WEST HAVEN, CT\n(41.272336, -72.949817)","New Haven, CT\n(41.308252, -72.924161)",Unknown/Missing
4918,08/05/2018,Date of death,28.0,Male,Black or African American,Hartford,Hartford,CT,Hartford,Hartford,NaN,Residence,Drug abuse,HARTFORD,HARTFORD,CT,Residence,NaN,"Multidrug Toxicity Including Cocaine, Fentanyl...",Accident,NaN,Y,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,Y,NaN,"HARTFORD, CT\n(41.765775, -72.673356)","HARTFORD, CT\n(41.765775, -72.673356)","Hartford, CT\n(41.765775, -72.673356)",Unknown/Missing
5989,06/15/2018,Date of death,27.0,Male,Black or African American,Windsor,Hartford,CT,Hartford,Hartford,NaN,In Vehicle,Substance Abuse,HARTFORD,HARTFORD,CT,Hospital,NaN,Complications of Acute and Chronic Opiate/Opio...,Accident,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,Y,NaN,"WINDSOR, CT\n(41.852781, -72.64379)","HARTFORD, CT\n(41.765775, -72.673356)","Hartford, CT\n(41.765775, -72.673356)",Hispanic/Latino
6283,06/16/2018,Date of death,26.0,Male,Black or African American,New haven,New haven,CT,West haven,New haven,NaN,Residence,Substance abuse,WEST HAVEN,NEW HAVEN,CT,Other,NaN,"ACUTE COMBINED FENTANYL, ACETYL FENTANYL, OXYC...",Accident,NaN,NaN,NaN,NaN,Y,Y,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,"NEW HAVEN, CT\n(41.308252, -72.924161)","WEST HAVEN, CT\n(41.272336, -72.949817)","West Haven, CT\n(41.272336, -72.949817)",Unknown/Missing
7455,11/02/2018,Date of death,25.0,Male,Black or African American,Windsor,Hartford,CT,,,NaN,Unknown,Substance Abuse,HARTFORD,HARTFORD,CT,Hospital,NaN,Acute Intoxication From the Combined Effects

#### what are the top 5 causes of death?

In [22]:
drug_deaths.groupby('Cause of Death').size().sort_values(ascending=False).head(5)

Cause of Death
Acute Fentanyl Intoxication    578
Acute Cocaine Intoxication     164
Multiple Drug Toxicity         131
Heroin Intoxication            130
Acute Heroin Intoxication      121
dtype: int64

#### list of people who were intoxicated by more than 3 drugs

In [23]:
# 1. creating a 'num_drugs' columns
# 2. creating a lambda function that defines the number of times a row has 'Y' value
# 3. outputting people who were intoxicated by over 3 drugs
drug_deaths['num_drugs'] = drug_deaths.apply(lambda x: (x == 'Y').sum(), axis=1)

In [24]:
drug_deaths[drug_deaths['num_drugs'] > 3]

,Date,Date Type,Age,Sex,Race,Residence City,Residence County,Residence State,Injury City,Injury County,Injury State,Injury Place,Description of Injury,Death City,Death County,Death State,Location,Location if Other,Cause of Death,Manner of Death,Other Significant Conditions,Heroin,Heroin death certificate (DC),Cocaine,Fentanyl,Fentanyl Analogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Meth/Amphetamine,Amphet,Tramad,Hydromorphone,Morphine (Not Heroin),Xylazine,Gabapentin,Opiate NOS,Heroin/Morph/Codeine,Other Opioid,Any Opioid,Other,ResidenceCityGeo,InjuryCityGeo,DeathCityGeo,Ethnicity_New,num_drugs
2,01/26/2022,Date of death,34.0,Male,White,Brookfield,Fairfield,CT,Brookfield,Fairfield,CT,Home,Substance Abuse,NaN,NaN,NaN,Decedent’s Home,NaN,Acute Intoxication due to the Combined Effects...,Accident,NaN,Y,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,Y,NaN,Y,NaN,"BROOKFIELD, CT\n(41.46736000000004, -73.398259...","BROOKFIELD, CT\n(41.46736000000004, -73.398259...","CT\n(41.57350273000003, -72.73830590799997)",Unknown/Missing,6
5,02/18/2022,Date of death,38.0,Male,White,Waterbury,New haven,CT,Waterbury,New haven,CT,Home,Drug use,NaN,NaN,NaN,Hospital - ER/Outpatient,NaN,Acute Intoxication Combined Effects of Ethanol...,Accident,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,Y,NaN,"WATERBURY, CT\n(41.55490000000003, -73.0464699...","WATERBURY, CT\n(41.55490000000003, -73.0464699...","CT\n(41.57350273000003, -72.73830590799997)",Unknown/Missing,5
9,04/16/2022,Date of death,24.0,Male,White,Bozrah,New london,CT,Bozrah,New london,CT,Home,Substance abuse,NaN,NaN,NaN,Decedent’s Home,NaN,Acute Intoxication by the Combined Effects of ...,Accident,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,"BOZRAH, CT\n(41.56752000000006, -72.1506099999...","BOZRAH, CT\n(41.56752000000006, -72.1506099999...","CT\n(41.57350273000003, -72.73830590799997)",Unknown/Missing,4
16,07/25/2023,Date of death,62.0,Female,White,Danbury,Fairfield,CT,Danbury,Fairfield,CT,Home,Substance abuse.,NaN,NaN,NaN,NaN,NaN,Acute Intoxication the Combined Effects of fen...,Accident,NaN,Y,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,Y,NaN,"DANBURY, CT\n(41.393666, -73.451539)","DANBURY, CT\n(41.393666, -73.451539)","CT\n(41.57350273, -72.738305908)",Unknown/Missing,5
17,08/11/2023,Date of death,32.0,Male,White,Brooklyn,Windham,CT,Brooklyn,Windham,CT,Home,Substance Use,NaN,NaN,NaN,NaN,NaN,Acute Intoxication by the Combined Effects of ...,Accident,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,Y,NaN,"BROOKLYN, CT\n(41.7882521, -71.9496283)","BROOKLYN, CT\n(41.7882521, -71.9496283)","CT\n(41.57350273, -72.738305908)",Unknown/Missing,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12953,03/10/2024,Date of death,49.0,Female,Black or African American,New haven,New haven,CT,New haven,New haven,CT,Other,Substance Use,NaN,NaN,NaN,NaN,NaN,Acute Intoxication by the Combined Effects of ...,Accident,NaN,NaN,NaN,Y,Y,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,"NEW HAVEN, CT\n(41.30801, -72.92432)","NEW HAVEN, CT\n(41.30801, -72.92432)","CT\n(41.76381, -72.67398)",Unknown/Missing,4
12956,05/28/2024,Date of death,41.0,Male,Black or African American,Bristol,Hartford,CT,Bristol,Hartford,CT,Home,Substance Use,NaN,NaN,NaN,NaN,NaN,Acute Intoxication by the Combined Effects of ...,Accident,NaN,NaN,NaN,Y,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,"BRISTOL, CT\n(41.67574, -72.94661)","BRISTOL, CT\n(41.67574, -72.94661)","CT\n(41.76381, -72.67398)",Non-Hispanic/Latino,5
12958,09/01/2024,Date of death,48.0,Male,Black or African American,Waterbury,New haven,CT,Waterbury,New haven,CT,Home,Substance Use,NaN,NaN,NaN,NaN,NaN,A

#### for top 5 counties by death count, what is the average age for men / women that died there?

In [25]:
# 1. outputting 5 counties with the highest death count and assigning it to top_counties_deaths

top_counties_deaths = drug_deaths.groupby('Death County').size().sort_values(ascending=False).head()

In [26]:
# 2. grouping only by 5 counties with the highest death count, then grouping by sex and outputting the average age
# 3. converting the series to dataframe (and renaming mean age column) to prepare it for an upcoming join

drug_deaths.groupby(['Death County', 'Sex'])['Age'].mean().round().loc[top_counties_deaths.index]
df_death_county_grouped = drug_deaths.groupby(['Death County', 'Sex'])['Age'].mean().round().loc[top_counties_deaths.index].rename('Mean Age').to_frame()

In [27]:
# 4. the purpose of the join is to output the number of deaths occurred in listed counties
#    converting top_counties_deaths to dataframe too and joining two dataframes


df_final = df_death_county_grouped.join(top_counties_deaths.rename('Total Deaths').to_frame(), on='Death County')
df_final

Mean Age  Total Deaths
Death County Sex                           
HARTFORD     Female      43.0          2441
             Male        44.0          2441
NEW HAVEN    Female      44.0          2406
             Male        44.0          2406
FAIRFIELD    Female      44.0          1278
             Male        43.0          1278
NEW LONDON   Female      41.0           726
             Male        42.0           726
LITCHFIELD   Female      43.0           418
             Male        41.0           418

In [28]:
# 5. finally, resetting index to fix column name misalignment

df_final.reset_index(inplace=True)
df_final

,Death County,Sex,Mean Age,Total Deaths
0,HARTFORD,Female,43.0,2441
1,HARTFORD,Male,44.0,2441
2,NEW HAVEN,Female,44.0,2406
3,NEW HAVEN,Male,44.0,2406
4,FAIRFIELD,Female,44.0,1278
5,FAIRFIELD,Male,43.0,1278
6,NEW LONDON,Female,41.0,726
7,NEW LONDON,Male,42.0,726
8,LITCHFIELD,Female,43.0,418
9,LITCHFIELD,Male,41.0,418


In [29]:
# but the following output is also possible:

drug_deaths.groupby(['Death County', 'Sex'])['Age'].mean().round().loc[top_counties_deaths.index]

Death County  Sex   
HARTFORD      Female    43.0
              Male      44.0
NEW HAVEN     Female    44.0
              Male      44.0
FAIRFIELD     Female    44.0
              Male      43.0
NEW LONDON    Female    41.0
              Male      42.0
LITCHFIELD    Female    43.0
              Male      41.0
Name: Age, dtype: float64